In [ ]:
# default_exp joint_entropy

In [ ]:
# hide
import blackhc.project.script

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


# Joint Entropies
> Computing joint entropies exactly or by using importance sampling

This module helps compute joint entropies for dependent categorical variables given via a density $p((y_i)_i|w))$ in the Bayesian setting. We compute the density $p((y_i)_i)$ by marginalizing over $w$.

Two cases are implemented:

* exact joint entropies (which works for up 5 to joint variables depending on memory and # of classes);
* estimated joint entropies using importance sampling of configurations.

Note: "exact" based on the given draws of $w$. They are still an approximation because we do not integrate over $w$ but use Monte-Carlo samples.

Number of inference samples `K`:

In [ ]:
K = 20

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# exports
import math
from typing import Optional

import torch
from toma import toma
from blackhc.progress_bar import create_progress_bar


def compute_conditional_entropy(log_probs_N_K_C: torch.Tensor) -> torch.Tensor:
    N, K, C = log_probs_N_K_C.shape

    entropies_N = torch.empty(N, dtype=torch.double)

    log_probs_N_K_C = log_probs_N_K_C.to(torch.double)

    pbar = create_progress_bar(
        N, tqdm_args=dict(desc="Conditional Entropy", leave=False)
    )
    pbar.start()

    @toma.execute.chunked(log_probs_N_K_C, 65536)
    def compute(log_probs_n_K_C, start: int, end: int):
        nats_n_K_C = log_probs_n_K_C * torch.exp(log_probs_n_K_C)
        nats_n_K_C[torch.isnan(nats_n_K_C)] = 0.0

        entropies_N[start:end].copy_(-torch.sum(nats_n_K_C, dim=(1, 2)) / K)
        pbar.update(end - start)

    pbar.finish()

    return entropies_N


def compute_entropy(log_probs_N_K_C: torch.Tensor) -> torch.Tensor:
    N, K, C = log_probs_N_K_C.shape

    entropies_N = torch.empty(N, dtype=torch.double)

    log_probs_N_K_C = log_probs_N_K_C.to(torch.double)

    pbar = create_progress_bar(N, tqdm_args=dict(desc="Entropy", leave=False))
    pbar.start()

    @toma.execute.chunked(log_probs_N_K_C, 65536)
    def compute(log_probs_n_K_C, start: int, end: int):
        mean_log_probs_n_C = torch.logsumexp(log_probs_n_K_C, dim=1) - math.log(K)
        nats_n_C = mean_log_probs_n_C * torch.exp(mean_log_probs_n_C)
        nats_n_C[torch.isnan(nats_n_C)] = 0.0

        entropies_N[start:end].copy_(-torch.sum(nats_n_C, dim=1))
        pbar.update(end - start)

    pbar.finish()

    return entropies_N

To run tests, we need a few sampled distributions.

In [ ]:
import numpy as np


def get_mixture_prob_dist(p1, p2, m):
    return (1.0 - m) * np.asarray(p1) + m * np.asarray(p2)


p1 = [0.1, 0.2, 0.2, 0.5]
p2 = [0.5, 0.2, 0.1, 0.2]
y1_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.6, 0.2, 0.1]
p2 = [0.0, 0.5, 0.5, 0.0]
y2_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]


def nested_to_tensor(*l):
    return torch.stack(list(map(torch.as_tensor, l)))


ys_ws = nested_to_tensor(y1_ws, y2_ws, y1_ws, y2_ws, y1_ws, y2_ws, y1_ws, y2_ws)

<ipython-input-6-a4f3ef56f743>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/torch/csrc/utils/tensor_new.cpp:201.)
  return torch.stack(list(map(torch.as_tensor, l)))


In [ ]:
# hide

p = [0.25, 0.25, 0.25, 0.25]
yu_ws = [p for m in range(K)]
yus_ws = nested_to_tensor(yu_ws, yu_ws, yu_ws, yu_ws)

# JointEntropy interface

Before we look at any implementations, we want to define an interface that we want to support.

In [ ]:
# export


class JointEntropy:
    """Random variables (all with the same # of categories $C$) can be added via `JointEntropy.add_variables`.

    `JointEntropy.compute` computes the joint entropy.

    `JointEntropy.compute_batch` computes the joint entropy of the added variables with each of the variables in the provided batch probabilities in turn."""

    def compute(self) -> torch.Tensor:
        """Computes the entropy of this joint entropy."""
        raise NotImplementedError()

    def add_variables(self, log_probs_N_K_C: torch.Tensor) -> "JointEntropy":
        """Expands the joint entropy to include more terms."""
        raise NotImplementedError()

    def compute_batch(
        self, log_probs_B_K_C: torch.Tensor, output_entropies_B=None
    ) -> torch.Tensor:
        """Computes the joint entropy of the added variables together with the batch (one by one)."""
        raise NotImplementedError()

    def lazy_compute_batch(
        self,
        log_probs_B_K_C: torch.Tensor,
        previous_entropies_B: torch.Tensor,
        top_k: int,
        output_entropies_B=None,
    ) -> torch.Tensor:
        """
        Computes the joint entropy of the added variables together with the batch (one by one) using a lazy-greedy approach.

        For this previous_entropies_B has to be sorted descending. We only support a top-k version here.

        The idea is that once the number of updated scores > largest not-updated score is larger than k, the top-k can
        only come from the updated scores.
        """
        raise NotImplementedError()

In [ ]:
show_doc(JointEntropy.add_variables)
show_doc(JointEntropy.compute)
show_doc(JointEntropy.compute_batch)

<h4 id="JointEntropy.add_variables" class="doc_header"><code>JointEntropy.add_variables</code><a href="__main__.py#L15" class="source_link" style="float:right">[source]</a></h4>

> <code>JointEntropy.add_variables</code>(**`log_probs_N_K_C`**:`Tensor`)

Expands the joint entropy to include more terms.

<h4 id="JointEntropy.compute" class="doc_header"><code>JointEntropy.compute</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>JointEntropy.compute</code>()

Computes the entropy of this joint entropy.

<h4 id="JointEntropy.compute_batch" class="doc_header"><code>JointEntropy.compute_batch</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>JointEntropy.compute_batch</code>(**`log_probs_B_K_C`**:`Tensor`, **`output_entropies_B`**=*`None`*)

Computes the joint entropy of the added variables together with the batch (one by one).

## Exact Joint Entropies

To compute exact joint entropies, we have to compute all possible configurations of the $y_i$ and evaluate_old $p(y_1, \dots, y_n)$ by averaging over $p(y_1, \dots, y_n|w)$.

The number of samples $M=C^N$, where $N$ is the number of variables in the joint and $C$ is the number of classes.

For this, we provide a class `ExactJointEntropy` that takes $K$ and starts with no variables in the joint.

### In the Paper
![Version in the paper](batchbald_exact_joint_entropy.png)

### Implementation

In [ ]:
# exports


class ExactJointEntropy(JointEntropy):
    joint_probs_M_K: torch.Tensor

    def __init__(self, joint_probs_M_K: torch.Tensor):
        self.joint_probs_M_K = joint_probs_M_K

    @staticmethod
    def empty(K: int, device=None, dtype=None) -> "ExactJointEntropy":
        return ExactJointEntropy(torch.ones((1, K), device=device, dtype=dtype))

    def compute(self) -> torch.Tensor:
        probs_M = torch.mean(self.joint_probs_M_K, dim=1, keepdim=False)
        nats_M = -torch.log(probs_M) * probs_M
        nats_M[torch.isnan(nats_M)] = 0.0
        del probs_M
        entropy = torch.sum(nats_M)
        del nats_M
        return entropy

    def add_variables(self, log_probs_N_K_C: torch.Tensor) -> "ExactJointEntropy":
        assert self.joint_probs_M_K.shape[1] == log_probs_N_K_C.shape[1]

        N, K, C = log_probs_N_K_C.shape
        joint_probs_K_M_1 = self.joint_probs_M_K.t()[:, :, None]

        probs_N_K_C = log_probs_N_K_C.exp()

        # Using lots of memory.
        for i in range(N):
            probs_i__K_1_C = probs_N_K_C[i][:, None, :].to(
                joint_probs_K_M_1, non_blocking=True
            )
            joint_probs_K_M_C = joint_probs_K_M_1 * probs_i__K_1_C
            joint_probs_K_M_1 = joint_probs_K_M_C.reshape((K, -1, 1))

        self.joint_probs_M_K = joint_probs_K_M_1.squeeze(2).t()
        return self

    def compute_batch(self, log_probs_B_K_C: torch.Tensor, output_entropies_B=None):
        return self.lazy_compute_batch(
            log_probs_B_K_C=log_probs_B_K_C,
            previous_entropies_B=None,
            top_k=None,
            output_entropies_B=output_entropies_B,
        )

    def lazy_compute_batch(
        self,
        log_probs_B_K_C: torch.Tensor,
        previous_entropies_B: Optional[torch.Tensor],
        top_k: Optional[int],
        output_entropies_B=None,
    ):
        assert self.joint_probs_M_K.shape[1] == log_probs_B_K_C.shape[1]

        B, K, C = log_probs_B_K_C.shape
        M = self.joint_probs_M_K.shape[0]

        if output_entropies_B is None:
            output_entropies_B = torch.empty(
                B, dtype=log_probs_B_K_C.dtype, device=log_probs_B_K_C.device
            )

        pbar = create_progress_bar(
            B, tqdm_args=dict(desc="ExactJointEntropy.compute_batch", leave=False)
        )
        pbar.start()

        copy_old_scores = None

        @toma.execute.chunked(log_probs_B_K_C, initial_step=1024, dimension=0)
        def chunked_joint_entropy(
            chunked_log_probs_b_K_C: torch.Tensor, start: int, end: int
        ):
            nonlocal copy_old_scores

            if start == 0:
                pbar.reset()
                copy_old_scores = False

            if not copy_old_scores:
                chunked_probs_b_K_C = chunked_log_probs_b_K_C.exp()
                b = chunked_probs_b_K_C.shape[0]

                probs_b_M_C = torch.empty(
                    (b, M, C),
                    dtype=self.joint_probs_M_K.dtype,
                    device=self.joint_probs_M_K.device,
                )
                for i in range(b):
                    torch.matmul(
                        self.joint_probs_M_K,
                        chunked_probs_b_K_C[i].to(
                            self.joint_probs_M_K, non_blocking=True
                        ),
                        out=probs_b_M_C[i],
                    )
                probs_b_M_C /= K

                nats_b_M_C = -torch.log(probs_b_M_C) * probs_b_M_C
                nats_b_M_C[torch.isnan(nats_b_M_C)] = 0.0

                output_entropies_B[start:end].copy_(
                    torch.sum(nats_b_M_C, dim=(1, 2)), non_blocking=True
                )

                # Early out (if there is anything left to do)?
                if end < B and previous_entropies_B is not None:
                    if (
                        torch.sum(output_entropies_B[:end] > previous_entropies_B[end])
                        >= top_k
                    ):
                        copy_old_scores = True
            else:
                # Just copy old scores
                output_entropies_B[start:end].copy_(
                    previous_entropies_B[start:end], non_blocking=True
                )

            pbar.update(end - start)

        pbar.finish()

        return output_entropies_B

In [ ]:
show_doc(ExactJointEntropy.empty)
show_doc(ExactJointEntropy.add_variables)
show_doc(ExactJointEntropy.compute)
show_doc(ExactJointEntropy.compute_batch)

<h4 id="ExactJointEntropy.empty" class="doc_header"><code>ExactJointEntropy.empty</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>ExactJointEntropy.empty</code>(**`K`**:`int`, **`device`**=*`None`*, **`dtype`**=*`None`*)



<h4 id="ExactJointEntropy.add_variables" class="doc_header"><code>ExactJointEntropy.add_variables</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>ExactJointEntropy.add_variables</code>(**`log_probs_N_K_C`**:`Tensor`)

Expands the joint entropy to include more terms.

<h4 id="ExactJointEntropy.compute" class="doc_header"><code>ExactJointEntropy.compute</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>ExactJointEntropy.compute</code>()

Computes the entropy of this joint entropy.

<h4 id="ExactJointEntropy.compute_batch" class="doc_header"><code>ExactJointEntropy.compute_batch</code><a href="__main__.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>ExactJointEntropy.compute_batch</code>(**`log_probs_B_K_C`**:`Tensor`, **`output_entropies_B`**=*`None`*)

Computes the joint entropy of the added variables together with the batch (one by one).

### Examples

In [ ]:
joint_entropy = ExactJointEntropy.empty(K, dtype=torch.double)
entropy = joint_entropy.add_variables(ys_ws[:4].log()).compute()
assert np.isclose(entropy, 4.6479, atol=0.1)
entropy

tensor(4.6479, dtype=torch.float64)

In [ ]:
joint_entropy = ExactJointEntropy.empty(K, dtype=torch.float)
entropy = joint_entropy.add_variables(ys_ws[:4].log()).compute()
assert np.isclose(entropy, 4.6479, atol=0.1)
entropy

tensor(4.6479)

In [ ]:
joint_entropy = ExactJointEntropy.empty(K, dtype=torch.float)
entropies = joint_entropy.add_variables(ys_ws[:4].log()).compute_batch(ys_ws.log())
assert np.allclose(entropies, [5.9735, 5.6362, 5.9735, 5.6362, 5.9735, 5.6362, 5.9735, 5.6362])
entropies

ExactJointEntropy.compute_batch:   0%|          | 0/8 [00:00<?, ?it/s]

tensor([5.9735, 5.6362, 5.9735, 5.6362, 5.9735, 5.6362, 5.9735, 5.6362],
       dtype=torch.float64)

In [ ]:
# hide
joint_entropy = ExactJointEntropy.empty(K, dtype=torch.double)
entropies = joint_entropy.add_variables(yus_ws.log()).compute()
print(entropies, np.log(4) * 4)
assert np.isclose(entropies, 5.5452, atol=0.1)

tensor(5.5452, dtype=torch.float64) 5.545177444479562


## Sampled Joint Entropies

To compute approximate joint entropies, we have to sample possible configurations of the $y_i$ from $p(y_1, \dots, y_n|w)$ stratified by $p(w)$ and evaluate_old $p(y_1, \dots, y_n)$ by averaging over $p(y_1, \dots, y_n|w)$.

The number of samples is $M$, so we use $\frac{M}{K}$ samples per $w$.

For this, we provide a class `SampledJointEntropy` that takes $K$ and $M$, and implements the 'JointEntropy' interface.

To sample, we need a few helper functions.

In [ ]:
# exports
def batch_multi_choices(probs_b_C, M: int):
    """
    probs_b_C: Ni... x C

    Returns:
        choices: Ni... x M
    """
    probs_B_C = probs_b_C.reshape((-1, probs_b_C.shape[-1]))

    # samples: Ni... x draw_per_xx
    choices = torch.multinomial(probs_B_C, num_samples=M, replacement=True)

    choices_b_M = choices.reshape(list(probs_b_C.shape[:-1]) + [M])
    return choices_b_M


def gather_expand(data, dim, index):
    if gather_expand.DEBUG_CHECKS:
        assert len(data.shape) == len(index.shape)
        assert all(dr == ir or 1 in (dr, ir) for dr, ir in zip(data.shape, index.shape))

    max_shape = [max(dr, ir) for dr, ir in zip(data.shape, index.shape)]
    new_data_shape = list(max_shape)
    new_data_shape[dim] = data.shape[dim]

    new_index_shape = list(max_shape)
    new_index_shape[dim] = index.shape[dim]

    data = data.expand(new_data_shape)
    index = index.expand(new_index_shape)

    result = torch.gather(data, dim, index)
    return result


gather_expand.DEBUG_CHECKS = False

### In the Paper
![Version in the paper](batchbald_importance_sampling.png)

### Implementation

In [ ]:
# exports


class SampledJointEntropy(JointEntropy):
    """Random variables (all with the same # of categories $C$) can be added via `SampledJointEntropy.add_variables`.

    `SampledJointEntropy.compute` computes the joint entropy.

    `SampledJointEntropy.compute_batch` computes the joint entropy of the added variables with each of the variables in the provided batch probabilities in turn."""

    sampled_joint_probs_M_K: torch.Tensor

    def __init__(self, sampled_joint_probs_M_K: torch.Tensor):
        self.sampled_joint_probs_M_K = sampled_joint_probs_M_K

    @staticmethod
    def empty(K: int, device=None, dtype=None) -> "SampledJointEntropy":
        return SampledJointEntropy(torch.ones((1, K), device=device, dtype=dtype))

    @staticmethod
    def sample(probs_N_K_C: torch.Tensor, M: int) -> "SampledJointEntropy":
        K = probs_N_K_C.shape[1]

        # S: num of samples per w
        S = M // K

        choices_N_K_S = batch_multi_choices(probs_N_K_C, S).long()

        expanded_choices_N_1_K_S = choices_N_K_S[:, None, :, :]
        expanded_probs_N_K_1_C = probs_N_K_C[:, :, None, :]

        probs_N_K_K_S = gather_expand(
            expanded_probs_N_K_1_C, dim=-1, index=expanded_choices_N_1_K_S
        )
        assert expanded_probs_N_K_1_C.data_ptr() != probs_N_K_K_S.data_ptr()

        # exp sum log seems necessary to avoid 0s?
        probs_K_K_S = torch.sum(probs_N_K_K_S.log_(), dim=0, keepdim=False).exp_()
        del probs_N_K_K_S
        samples_K_M = probs_K_K_S.reshape((K, -1))

        samples_M_K = samples_K_M.t()
        return SampledJointEntropy(samples_M_K)

    def compute(self) -> torch.Tensor:
        sampled_joint_probs_M = torch.mean(
            self.sampled_joint_probs_M_K, dim=1, keepdim=False
        )
        nats_M = -sampled_joint_probs_M.log_()
        nats_M[torch.isnan(nats_M)] = 0.0
        entropy = torch.mean(nats_M)
        return entropy

    def add_variables(
        self, log_probs_N_K_C: torch.Tensor, M2: int
    ) -> "SampledJointEntropy":
        assert self.sampled_joint_probs_M_K.shape[1] == log_probs_N_K_C.shape[1]
        K = log_probs_N_K_C.shape[1]

        sample_K_M1_1 = self.sampled_joint_probs_M_K.t()[:, :, None]

        new_sample_M2_K = self.sample(log_probs_N_K_C.exp(), M2).sampled_joint_probs_M_K
        new_sample_K_1_M2 = new_sample_M2_K.t()[:, None, :]

        merged_sample_K_M1_M2 = sample_K_M1_1 * new_sample_K_1_M2
        merged_sample_K_M = merged_sample_K_M1_M2.reshape((K, -1))

        self.sampled_joint_probs_M_K = merged_sample_K_M.t()

        return self

    def compute_batch(self, log_probs_B_K_C: torch.Tensor, output_entropies_B=None):
        return self.lazy_compute_batch(
            log_probs_B_K_C=log_probs_B_K_C,
            previous_entropies_B=None,
            top_k=None,
            output_entropies_B=output_entropies_B,
        )

    def lazy_compute_batch(
        self,
        log_probs_B_K_C: torch.Tensor,
        previous_entropies_B: Optional[torch.Tensor],
        top_k: Optional[int],
        output_entropies_B=None,
    ):
        assert self.sampled_joint_probs_M_K.shape[1] == log_probs_B_K_C.shape[1]

        B, K, C = log_probs_B_K_C.shape
        M = self.sampled_joint_probs_M_K.shape[0]

        if output_entropies_B is None:
            output_entropies_B = torch.empty(
                B, dtype=log_probs_B_K_C.dtype, device=log_probs_B_K_C.device
            )

        pbar = create_progress_bar(
            B, tqdm_args=dict(desc="SampledJointEntropy.compute_batch", leave=False)
        )
        pbar.start()

        copy_old_scores = None

        @toma.execute.chunked(log_probs_B_K_C, initial_step=1024, dimension=0)
        def chunked_joint_entropy(
            chunked_log_probs_b_K_C: torch.Tensor, start: int, end: int
        ):
            nonlocal copy_old_scores

            if start == 0:
                pbar.reset()
                copy_old_scores = False

            if not copy_old_scores:
                b = chunked_log_probs_b_K_C.shape[0]

                probs_b_M_C = torch.empty(
                    (b, M, C),
                    dtype=self.sampled_joint_probs_M_K.dtype,
                    device=self.sampled_joint_probs_M_K.device,
                )
                for i in range(b):
                    torch.matmul(
                        self.sampled_joint_probs_M_K,
                        chunked_log_probs_b_K_C[i]
                        .to(self.sampled_joint_probs_M_K, non_blocking=True)
                        .exp(),
                        out=probs_b_M_C[i],
                    )
                probs_b_M_C /= K

                q_1_M_1 = self.sampled_joint_probs_M_K.mean(dim=1, keepdim=True)[None]

                output_entropies_B[start:end].copy_(
                    torch.sum(
                        -torch.log(probs_b_M_C) * probs_b_M_C / q_1_M_1, dim=(1, 2)
                    )
                    / M,
                    non_blocking=True,
                )

                # Early out (if there is anything left to do)?
                if end < B and previous_entropies_B is not None:
                    if (
                        torch.sum(output_entropies_B[:end] > previous_entropies_B[end])
                        >= top_k
                    ):
                        copy_old_scores = True
            else:
                # Just copy old scores
                output_entropies_B[start:end].copy_(
                    previous_entropies_B[start:end], non_blocking=True
                )

            pbar.update(end - start)

        pbar.finish()

        return output_entropies_B

In [ ]:
show_doc(SampledJointEntropy.empty)
show_doc(SampledJointEntropy.sample)
show_doc(SampledJointEntropy.compute)
show_doc(SampledJointEntropy.add_variables)
show_doc(SampledJointEntropy.compute_batch)

<h4 id="SampledJointEntropy.empty" class="doc_header"><code>SampledJointEntropy.empty</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>SampledJointEntropy.empty</code>(**`K`**:`int`, **`device`**=*`None`*, **`dtype`**=*`None`*)



<h4 id="SampledJointEntropy.sample" class="doc_header"><code>SampledJointEntropy.sample</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>SampledJointEntropy.sample</code>(**`probs_N_K_C`**:`Tensor`, **`M`**:`int`)



<h4 id="SampledJointEntropy.compute" class="doc_header"><code>SampledJointEntropy.compute</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>SampledJointEntropy.compute</code>()

Computes the entropy of this joint entropy.

<h4 id="SampledJointEntropy.add_variables" class="doc_header"><code>SampledJointEntropy.add_variables</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>SampledJointEntropy.add_variables</code>(**`log_probs_N_K_C`**:`Tensor`, **`M2`**:`int`)

Expands the joint entropy to include more terms.

<h4 id="SampledJointEntropy.compute_batch" class="doc_header"><code>SampledJointEntropy.compute_batch</code><a href="__main__.py#L66" class="source_link" style="float:right">[source]</a></h4>

> <code>SampledJointEntropy.compute_batch</code>(**`log_probs_B_K_C`**:`Tensor`, **`output_entropies_B`**=*`None`*)

Computes the joint entropy of the added variables together with the batch (one by one).

### Examples

In [ ]:
joint_entropy = SampledJointEntropy.empty(K, dtype=torch.double)
entropy = joint_entropy.add_variables(ys_ws[:4].log(), 100000).compute()

print(entropy)
assert np.isclose(entropy, 4.6479, atol=0.1)

tensor(4.6510, dtype=torch.float64)


In [ ]:
joint_entropy = SampledJointEntropy.empty(K, dtype=torch.double)
entropy = joint_entropy.add_variables(ys_ws[:4].log(), 100000).compute()

print(entropy)
assert np.isclose(entropy, 4.6479, atol=0.1)

tensor(4.6476, dtype=torch.float64)


In [ ]:
joint_entropy = SampledJointEntropy.empty(K, dtype=torch.float)
entropies = joint_entropy.add_variables(ys_ws[:4].log(), 1000000).compute_batch(ys_ws.log())

print(entropies)
assert np.allclose(entropies, [5.9735, 5.6362, 5.9735, 5.6362, 5.9735, 5.6362, 5.9735, 5.6362], atol=0.05)

SampledJointEntropy.compute_batch:   0%|          | 0/8 [00:00<?, ?it/s]

tensor([5.9729, 5.6356, 5.9729, 5.6356, 5.9729, 5.6356, 5.9729, 5.6356],
       dtype=torch.float64)


## Dynamically chooses JointEntropy Method

Finally, we want to be able to dynamically pick either class depending on the maximum number of samples we want. (And also resample if necessary as we add variables.)

In [ ]:
# exports


class DynamicJointEntropy(JointEntropy):
    inner: JointEntropy
    log_probs_max_N_K_C: torch.Tensor
    N: int
    M: int

    def __init__(self, M: int, max_N: int, K: int, C: int, dtype=None, device=None):
        self.M = M
        self.N = 0
        self.max_N = max_N

        self.inner = ExactJointEntropy.empty(K, dtype=dtype, device=device)
        self.log_probs_max_N_K_C = torch.empty(
            (max_N, K, C), dtype=dtype, device=device
        )

    def add_variables(self, log_probs_N_K_C: torch.Tensor) -> "DynamicJointEntropy":
        C = self.log_probs_max_N_K_C.shape[2]
        add_N = log_probs_N_K_C.shape[0]

        assert self.log_probs_max_N_K_C.shape[0] >= self.N + add_N
        assert self.log_probs_max_N_K_C.shape[2] == C

        self.log_probs_max_N_K_C[self.N : self.N + add_N] = log_probs_N_K_C
        self.N += add_N

        num_exact_samples = C ** self.N
        if num_exact_samples > self.M:
            self.inner = SampledJointEntropy.sample(
                self.log_probs_max_N_K_C[: self.N].exp(), self.M
            )
        else:
            self.inner.add_variables(log_probs_N_K_C)

        return self

    def compute(self) -> torch.Tensor:
        return self.inner.compute()

    def compute_batch(
        self, log_probs_B_K_C: torch.Tensor, output_entropies_B=None
    ) -> torch.Tensor:
        return self.inner.compute_batch(log_probs_B_K_C, output_entropies_B)

    def lazy_compute_batch(
        self,
        log_probs_B_K_C: torch.Tensor,
        previous_entropies_B: Optional[torch.Tensor],
        top_k: Optional[int],
        output_entropies_B=None,
    ):
        return self.inner.lazy_compute_batch(
            log_probs_B_K_C=log_probs_B_K_C,
            previous_entropies_B=previous_entropies_B,
            top_k=top_k,
            output_entropies_B=output_entropies_B,
        )

In [ ]:
show_doc(DynamicJointEntropy.add_variables)
show_doc(DynamicJointEntropy.compute_batch)

<h4 id="DynamicJointEntropy.add_variables" class="doc_header"><code>DynamicJointEntropy.add_variables</code><a href="__main__.py#L18" class="source_link" style="float:right">[source]</a></h4>

> <code>DynamicJointEntropy.add_variables</code>(**`log_probs_N_K_C`**:`Tensor`)

Expands the joint entropy to include more terms.

<h4 id="DynamicJointEntropy.compute_batch" class="doc_header"><code>DynamicJointEntropy.compute_batch</code><a href="__main__.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>DynamicJointEntropy.compute_batch</code>(**`log_probs_B_K_C`**:`Tensor`, **`output_entropies_B`**=*`None`*)

Computes the joint entropy of the added variables together with the batch (one by one).

### Examples

In [ ]:
joint_entropy = DynamicJointEntropy(256, 8, K, 4, dtype=torch.double)
entropy = joint_entropy.add_variables(ys_ws[:4].log()).compute()

print(entropy)
assert np.isclose(entropy, 4.6479, atol=0.1)

tensor(4.6479, dtype=torch.float64)


In [ ]:
assert type(joint_entropy.inner) == ExactJointEntropy

In [ ]:
entropy = joint_entropy.add_variables(ys_ws[4:].log()).compute()

print(entropy)
assert np.isclose(entropy, 9.2756, atol=0.5)

tensor(9.1174, dtype=torch.float64)


In [ ]:
assert type(joint_entropy.inner) == SampledJointEntropy